In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [27]:
# Открыть HTML-файл и прочитать его содержимое
with open("malls.html", "r", encoding="utf-8") as file:
    html = file.read()

# Создаем объект BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Ищем все теги <img> с классом "img-with-alt"
img_tags = soup.find_all("img", class_="img-with-alt")

# Извлекаем значения атрибута alt
alt_texts = [img["alt"] for img in img_tags if "alt" in img.attrs]

# Создаем DataFrame
df = pd.DataFrame(alt_texts, columns=["Alt Text"])

# Сохраняем в CSV файл
df.to_csv("alt_texts.csv", index=False)

print("Data saved to 'alt_texts.csv'.")


Data saved to 'alt_texts.csv'.


In [51]:
df

,Alt Text
0,"Ригамолл (Московская область, городской округ ..."
1,"Галерея Ремонта (Москва, поселение Московский,..."
2,"Савёловский Модный (ул. Сущёвский Вал, 5, стр...."
3,"Family Room (Москва, поселение Московский, Кие..."
4,"Декстер (МКАД, 78-й километр, 14, корп. 1), то..."
...,...
903,"Торговый центр (Королёв, просп. Космонавтов, 3..."
904,"Галина (ул. Авиаторов, 18), торговый центр в М..."
905,"Торговый центр (ул. Введенского, 16), торговый..."
906,"Мини-ТЦ (Балашиха, микрорайон Кучино, просп. Ж..."


In [52]:
# Функция для извлечения названия и адреса
def extract_name_and_address(text):
    match = re.match(r"^(.*?)\s+\((.*?)\)", text)
    if match:
        name = match.group(1)  # Название до скобок
        address = match.group(2)  # Адрес в скобках
        return name, address
    return None, None

# Применяем функцию к каждой строке
df["Name"], df["Address"] = zip(*df["Alt Text"].apply(extract_name_and_address))

# Сохраняем результат в новый CSV
df.to_csv("parsed_malls.csv", index=False)

df


,Alt Text,Name,Address
0,"Ригамолл (Московская область, городской округ ...",Ригамолл,"Московская область, городской округ Красногорс..."
1,"Галерея Ремонта (Москва, поселение Московский,...",Галерея Ремонта,"Москва, поселение Московский, МКАД, 47-й килом..."
2,"Савёловский Модный (ул. Сущёвский Вал, 5, стр....",Савёловский Модный,"ул. Сущёвский Вал, 5, стр. 1А"
3,"Family Room (Москва, поселение Московский, Кие...",Family Room,"Москва, поселение Московский, Киевское шоссе, ..."
4,"Декстер (МКАД, 78-й километр, 14, корп. 1), то...",Декстер,"МКАД, 78-й километр, 14, корп. 1"
...,...,...,...
903,"Торговый центр (Королёв, просп. Космонавтов, 3...",Торговый центр,"Королёв, просп. Космонавтов, 36Б"
904,"Галина (ул. Авиаторов, 18), торговый центр в М...",Галина,"ул. Авиаторов, 18"
905,"Торговый центр (ул. Введенского, 16), торговый...",Торговый центр,"ул. Введенского, 16"
906,"Мини-ТЦ (Балашиха, микрорайон Кучино, просп. Ж...",Мини-ТЦ,"Балашиха, микрорайон Кучино, просп. Жуковского, 5"


In [54]:
# Удаляем лишний столбец
malls = df.drop(columns=["Alt Text"])

# Меняем порядок столбцов
malls = malls[["Name", "Address"]]  в

# Функция для добавления "Москва"
def add_moscow_if_missing(address):
    if "Москва" not in address:
        return f"Москва, {address}"
    return address

# Применяем функцию к столбцу "Address"
malls["Address"] = malls["Address"].apply(add_moscow_if_missing)

# Сохраняем результат в CSV
#malls.to_csv("malls_coord.csv", index=False)

malls

,Name,Address
0,Ригамолл,"Москва, Московская область, городской округ Кр..."
1,Галерея Ремонта,"Москва, поселение Московский, МКАД, 47-й килом..."
2,Савёловский Модный,"Москва, ул. Сущёвский Вал, 5, стр. 1А"
3,Family Room,"Москва, поселение Московский, Киевское шоссе, ..."
4,Декстер,"Москва, МКАД, 78-й километр, 14, корп. 1"
...,...,...
903,Торговый центр,"Москва, Королёв, просп. Космонавтов, 36Б"
904,Галина,"Москва, ул. Авиаторов, 18"
905,Торговый центр,"Москва, ул. Введенского, 16"
906,Мини-ТЦ,"Москва, Балашиха, микрорайон Кучино, просп. Жу..."


In [49]:
# Ключ API
YANDEX_API_KEY = "_"

# Базовый URL для Geocoder API
GEOCODER_URL = "https://geocode-maps.yandex.ru/1.x/"

# Функция для получения координат по запросу
def get_coordinates(query, apikey, city="Москва"):
    params = {
        "apikey": apikey,
        "geocode": f"{city}, {query}",  # Запрос с уточнением города
        "format": "json",              # Результат в формате JSON
    }
    response = requests.get(GEOCODER_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        try:
            # Получение координат первого найденного объекта
            pos = (
                data["response"]["GeoObjectCollection"]["featureMember"][0]
                ["GeoObject"]["Point"]["pos"]
            )
            # Координаты возвращаются в виде строки "долгота широта", преобразуем их
            lon, lat = map(float, pos.split())
            return lat, lon
        except (IndexError, KeyError):
            print(f"Не удалось найти объект: {query}")
            return None
    else:
        print(f"Ошибка {response.status_code}: {response.text}")
        return None

# Чтение данных из CSV-файла
#df = pd.read_csv("alt_texts.csv")

# Получение координат для каждого торгового центра и добавление их в новый столбец
coordinates = []
for mall in malls['Address']:
    coords = get_coordinates(mall, YANDEX_API_KEY)
    if coords:
        coordinates.append(coords)
    else:
        coordinates.append((None, None))  # Если координаты не найдены, добавляем None

# Добавляем координаты в DataFrame
malls["Coordinates"] = coordinates

# Сохраняем результат в новый CSV-файл
malls.to_csv("malls_with_coordinates.csv", index=False)

#print("Results saved to 'malls_with_coordinates.csv'.")

In [5]:
malls

,Name,Address,Coordinates
0,Ригамолл,"Москва, Московская область, городской округ Кр...","(55.799699, 37.273967)"
1,Галерея Ремонта,"Москва, поселение Московский, МКАД, 47-й килом...","(55.658764, 37.429986)"
2,Савёловский Модный,"Москва, ул. Сущёвский Вал, 5, стр. 1А","(55.794143, 37.592833)"
3,Family Room,"Москва, поселение Московский, Киевское шоссе, ...","(55.629836, 37.42412)"
4,Декстер,"Москва, МКАД, 78-й километр, 14, корп. 1","(55.893112, 37.501223)"
...,...,...,...
903,Торговый центр,"Москва, Королёв, просп. Космонавтов, 36Б","(55.90795, 37.865966)"
904,Галина,"Москва, ул. Авиаторов, 18","(55.644689, 37.400099)"
905,Торговый центр,"Москва, ул. Введенского, 16","(55.642484, 37.540425)"
906,Мини-ТЦ,"Москва, Балашиха, микрорайон Кучино, просп. Жу...","(55.752789, 37.957073)"
